In [20]:
pip install pytrec_eval

Note: you may need to restart the kernel to use updated packages.


In [22]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/xyz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
from nltk.corpus import wordnet as wn

In [2]:
from nltk.corpus import wordnet as wn
dictionary=[]
for synset in wn.all_synsets():
    for lemma in synset.lemmas():
        dictionary.append(lemma.name())

In [8]:
with open('Birkbeck_Subset.txt', 'r') as file:
    lines = file.readlines()
lines = [line.strip() for line in lines if line.strip()]
birkbeck_corpus=[]
original_word=''
for text in lines:
    if text.startswith('$'):
        original_word=text[1:]
    else:
#         print(text)
        birkbeck_corpus.append((text,original_word))

In [4]:
# def MED(str1, str2, strlen1, strlen2):
#     if strlen1 == 0:
#         return strlen2
#     if strlen2 == 0:
#         return strlen1
#     if str1[strlen1-1] == str2[strlen2-1]:
#         return MED(str1, str2, strlen1-1, strlen2-1)
#     return 1 + min(MED(str1, str2, strlen1, strlen2-1),
#                    MED(str1, str2, strlen1-1, strlen2),
#                    MED(str1, str2, strlen1-1, strlen2-1))

In [5]:
def min_edit_distance(str1, str2):
    m = len(str1)
    n = len(str2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i][j - 1],dp[i - 1][j],dp[i - 1][j - 1])
    return dp[m][n]



In [7]:
from difflib import SequenceMatcher
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
def findRelatedWord(i):
    top_k_items=[]
#     filtered_dictionary=[item for item in dictionary if item[0]==i[0][0]]
    filtered_dictionary=list(dict.fromkeys(dictionary))
    for filtered_item in filtered_dictionary:
        ratio = min_edit_distance(i[0], filtered_item)
        if len(top_k_items) < 10 or ratio < top_k_items[-1][1]:
            top_k_items.append((filtered_item, ratio))
            top_k_items.sort(key=lambda x: x[1], reverse=False)
            top_k_items = top_k_items[:10]
    return { 
        'related_word':top_k_items,
        'original_word':i[1]
                               }
ks = [1, 5, 10]
satk_values = defaultdict(list)
with ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(findRelatedWord, i) for i in birkbeck_corpus]
    for future in as_completed(futures):
        result = future.result()
        related_words = result['related_word']
        original_word = result['original_word']
        for k in ks:
            is_correct_in_top_k = original_word in [word[0] for word in related_words[:k]]
            satk_values[k].append(1 if is_correct_in_top_k else 0)
average_satk_values = {k: sum(satk_values[k]) / len(satk_values[k]) for k in ks}
for k, avg_satk in average_satk_values.items():
    print(f"Average s@{k}: {avg_satk}")

Average s@1: 0.302
Average s@5: 0.43
Average s@10: 0.482
